<a href="https://colab.research.google.com/github/ykhier/Cloud-Computing-Cat/blob/main/HW2_CAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports


In [ ]:
# ===== Standard Library =====
import os
import io
import re
import json
import time
import uuid
import asyncio
import threading
import warnings
import random
from collections import defaultdict
from datetime import datetime, date, timezone
from threading import Timer
from zoneinfo import ZoneInfo
import base64
from PIL import Image

# ===== Third-Party Libraries =====
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot
from nltk.stem import PorterStemmer
from transformers import pipeline

# ===== Jupyter / Widgets =====
import ipywidgets as widgets
from ipywidgets import VBox, Output
from IPython.display import display, HTML, clear_output

# ===== Firebase =====
import firebase_admin
from firebase_admin import credentials, storage, db

#CSS


In [ ]:
def load_cat_css():
    display(HTML("""
    <style>
    /* ===================== HEADER ===================== */

    /* Header container */
    .cat-header{
        text-align: center;
        margin-top: 10px;
    }

    /* Header title text */
    .cat-header-title{
        font-size: 26px;
        font-weight: 700;
        color: #1a73e8;
        letter-spacing: 0.03em;
        margin-bottom: 20px;
        font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
    }

    /* ===================== TABS ===================== */

    /* Tabs wrapper */
    .custom-tabs{
        margin-top: 0;
        margin-left: auto;
        margin-right: auto;
        width: 100%;
        max-width: 1200px;
        border: none !important;
        box-shadow: none !important;
        background: #ffffff !important;
        font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
    }

    /* Tabs bar */
    .custom-tabs .p-TabBar{
        background: #ffffff !important;
        border: none !important;
        border-bottom: 1px solid #e5e5e5 !important;
        box-shadow: none !important;
        padding: 0 40px !important;
    }

    /* Single tab */
    .custom-tabs .p-TabBar-tab{
        position: relative;
        flex: 0 0 auto;
        text-align: center;
        font-size: 15px;
        font-weight: 500;
        color: #5f6368;
        background: transparent !important;
        border: none !important;
        box-shadow: none !important;
        border-radius: 0 !important;
        padding: 10px 0 !important;
        margin: 0 24px 0 0 !important;
        border-bottom: 2px solid transparent !important;
        cursor: pointer;
        transition: color 0.15s ease-out, border-bottom-color 0.15s ease-out;
        min-height: auto !important;
    }

    /* Tab hover */
    .custom-tabs .p-TabBar-tab:hover{
        color: #1a73e8;
    }

    /* Active tab */
    .custom-tabs .p-TabBar-tab.p-mod-current{
        color: #1a73e8 !important;
        border-bottom-color: #4285f4 !important;
        background: transparent !important;
    }

    /* Tabs content wrapper */
    .custom-tabs .widget-tab-contents{
        background: #ffffff !important;
        border: none !important;
        box-shadow: none !important;
    }

    /* Output inside tabs */
    .custom-tabs .widget-output{
        padding: 16px 40px 24px 40px;
        background: #ffffff;
        color: #202124;
        font-size: 13px;
        border: none !important;
    }

    /* ===================== UPLOAD PHOTO ===================== */

    /* Upload form wrapper */
    .plant-form{
        max-width: 520px;
        margin: 0 auto;
        padding-top: 10px;
        font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
    }

    /* Upload title */
    .plant-form-title{
        font-size: 20px;
        font-weight: 600;
        margin: 0 0 4px 0;
        color: #4285f4;
    }

    /* Upload subtitle */
    .plant-form-sub{
        font-size: 12px;
        color: #5f6368;
        margin-bottom: 14px;
    }

    /* Upload drop box */
    .plant-upload-box{
        border: 2px dashed #4a90e2;
        border-radius: 12px;
        width: 100%;
        height: 190px;
        display: flex;
        align-items: center;
        justify-content: center;
        background: #f8fbff;
        margin-bottom: 18px;
    }

    /* Upload button label */
    .plant-upload-box .widget-upload > label{
        display: inline-block;
        padding: 8px 26px;
        border-radius: 999px;
        background: #1a73e8;
        color: #ffffff;
        font-size: 13px;
        font-weight: 600;
        border: none;
        cursor: pointer;
        box-shadow: 0 3px 8px rgba(37, 99, 235, 0.30);
    }

    /* Upload button hover */
    .plant-upload-box .widget-upload > label:hover{
        filter: brightness(1.05);
    }

    /* Input label */
    .plant-label{
        font-size: 13px;
        font-weight: 500;
        color: #3c4043;
        margin-bottom: 4px;
    }

    /* Input container */
    .plant-input{
        margin-bottom: 10px;
    }

    /* Text input */
    .plant-input .widget-text input{
        width: 100%;
        border-radius: 6px;
        border: 1px solid #d1d5db !important;
        padding: 7px 10px;
        font-size: 13px;
        box-shadow: none !important;
    }

    /* Start button container */
    .plant-start-btn .widget-button{
        width: 100% !important;
        border-radius: 6px !important;
        font-weight: 600;
        font-size: 14px;
        border: 1px solid #1a73e8 !important;
        color: #ffffff !important;
        background: #1a73e8 !important;
        box-shadow: none !important;
        margin-top: 4px;
    }

    /* Start button hover */
    .plant-start-btn .widget-button:hover{
        filter: brightness(1.04);
    }

    /* Message container */
    .plant-message{
        margin-top: 10px;
        font-size: 13px;
        font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
    }

    /* Error message style */
    .plant-message-error{
        background: #fef2f2;
        color: #b91c1c;
        border: 1px solid #fecaca;
        border-radius: 8px;
        padding: 8px 10px;
    }

    /* Success message style */
    .plant-message-success{
        background: #ecfdf3;
        color: #166534;
        border: 1px solid #bbf7d0;
        border-radius: 8px;
        padding: 8px 10px;
    }

    /* ===================== IOT SENSORS ===================== */

    /* IoT main container */
    .iot-container{
        max-width: 1200px;
        margin: 0 auto;
        padding: 20px;
        background: #ffffff;
        font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
    }

    /* IoT header row */
    .iot-header{
        display: flex;
        justify-content: space-between;
        align-items: center;
        margin-bottom: 20px;
    }

    /* IoT title */
    .iot-title{
        font-size: 32px;
        font-weight: 600;
        margin: 0 0 8px 0;
        color: #4285f4;
    }

    /* Sensor card */
    .sensor-card{
        background: linear-gradient(135deg, #4285f4 0%, #6fa8f5 100%);
        border-radius: 20px;
        padding: 20px;
        margin-bottom: 20px;
        box-shadow: 0 4px 12px rgba(66, 133, 244, 0.3);
    }

    /* Sensor name */
    .sensor-name{
        color: #ffffff;
        font-size: 22px;
        font-weight: 600;
        margin-bottom: 20px;
        text-shadow: 0 2px 4px rgba(0, 0, 0, 0.2);
    }

    /* Metrics row */
    .metrics-container{
        display: flex;
        justify-content: space-around;
        gap: 5px;
        min-width: 450px;
    }

    /* Metric card */
    .metric-item{
        display: flex;
        flex-direction: column;
        align-items: center;
        width: 160px;
        padding: 18px 15px;
        border-radius: 12px;
        background: rgba(255, 255, 255, 0.9);
        backdrop-filter: blur(10px);
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1);
    }

    /* Circular progress wrapper */
    .circular-progress{
        position: relative;
        width: 110px;
        height: 110px;
        margin-bottom: 12px;
    }

    /* Progress ring rotation */
    .progress-ring{
        transform: rotate(-90deg);
    }

    /* Progress ring circle */
    .progress-ring__circle{
        transition: stroke-dashoffset 0.8s ease-in-out;
    }

    /* Background circle */
    .bg-circle{
        stroke: #e8eaed;
    }

    /* Value inside ring */
    .metric-value{
        position: absolute;
        top: 50%;
        left: 50%;
        transform: translate(-50%, -50%);
        color: #202124;
        font-size: 16px;
        font-weight: 700;
    }

    /* Metric label */
    .metric-label{
        color: #4285f4;
        font-size: 20px;
        font-weight: 700;
        text-align: center;
        margin-top: 6px;
    }

    /* Metric icon */
    .metric-icon{
        font-size: 24px;
        margin-bottom: 10px;
    }

    /* Status colors */
    .status-optimal, .stroke-optimal{ color: #22B14C; stroke: #22B14C; }
    .status-warning, .stroke-warning{ color: #ea8600; stroke: #ea8600; }
    .status-critical, .stroke-critical{ color: #d93025; stroke: #d93025; }

    /* Refresh button wrapper */
    .refresh-btn-container{
        display: flex !important;
        justify-content: center !important;
        width: 100% !important;
        margin-bottom: 20px !important;
    }

    /* Refresh button */
    .refresh-btn-container .widget-button{
        background: linear-gradient(135deg, #00d084 0%, #00b894 100%) !important;
        border: none !important;
        border-radius: 12px !important;
        color: white !important;
        font-size: 16px !important;
        font-weight: 600 !important;
        padding: 15px 40px !important;
        box-shadow: 0 4px 12px rgba(0, 208, 132, 0.3) !important;
        transition: all 0.2s ease !important;
        margin-bottom: 20px !important;
        min-width: 180px !important;
        min-height: 50px !important;
        line-height: 1.2 !important;
    }

    /* Refresh button hover */
    .refresh-btn-container .widget-button:hover{
        transform: translateY(-2px) !important;
        box-shadow: 0 6px 16px rgba(0, 208, 132, 0.4) !important;
        background: linear-gradient(135deg, #00b894 0%, #009977 100%) !important;
    }

    /* Refresh button active */
    .refresh-btn-container .widget-button:active{
        transform: translateY(0px) !important;
    }

    /* Graph container */
    .graph-container{
        background: linear-gradient(135deg, #4285f4 0%, #6fa8f5 100%);
        border-radius: 20px;
        padding: 20px;
        margin-top: 20px;
        box-shadow: 0 4px 12px rgba(66, 133, 244, 0.3);
    }

    /* Graph title */
    .graph-title{
        color: #ffffff;
        font-size: 22px;
        font-weight: 600;
        margin-bottom: 20px;
        text-shadow: 0 2px 4px rgba(0, 0, 0, 0.2);
        text-align: left;
    }

    /* Graphs row */
    .graphs-container{
        display: flex;
        justify-content: space-around;
        gap: 20px;
    }

    /* Single graph card */
    .single-graph-container{
        background: rgba(255, 255, 255, 0.95);
        border-radius: 12px;
        padding: 15px;
        flex: 1;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    }

    /* ===================== DASHBOARD ===================== */

    /* Dashboard wrapper */
    .dash-container{
        max-width: 1200px;
        margin: 0 auto;
        padding: 20px;
        font-family: system-ui, -apple-system, "Segoe UI", sans-serif;
    }

    /* Dashboard title */
    .dash-title{
        font-size: 32px;
        font-weight: 700;
        color: #1a73e8;
        margin-bottom: 16px;
    }

    /* Dashboard main card */
    .dash-card{
        background: linear-gradient(135deg, #7bb0ff 0%, #b2d0ff 100%);
        border-radius: 18px;
        box-shadow: 0 4px 10px rgba(0,0,0,0.15);
        padding: 18px;
    }

    /* Dashboard averages row */
    .dash-avg-row{
        margin-bottom: 6px;
    }

    /* Weather week row */
    .dash-weather-week{
        margin-top: 8px;
        display: flex;
        gap: 12px !important;
        justify-content: flex-start;
        width: 100%;
    }

    /* Weather main row */
    .dash-weather-main{
        display: flex;
        align-items: center;
        gap: 10px;
        margin: 6px 0 10px 0;
    }

    /* Weather icon */
    .dash-weather-icon{
        font-size: 34px;
        line-height: 1;
    }

    /* Weather temperature */
    .dash-weather-temp{
        font-size: 42px;
        font-weight: 800;
        letter-spacing: -1px;
        margin-bottom: 6px;
    }

    /* Weather subtext */
    .dash-subtext{
        font-size: 12px;
        margin-bottom: 4px;
    }

    /* Forecast pill */
    .dash-weather-week > div{
        background: rgba(255,255,255,0.35);
        border-radius: 12px;
        padding: 10px 6px !important;
        text-align: center;
        min-width: 64px !important;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }

    /* Forecast day label */
    .dash-forecast-day{
        font-size: 13px !important;
        font-weight: 700 !important;
    }

    /* Forecast temp label */
    .dash-forecast-temp{
        font-size: 15px !important;
        font-weight: 800 !important;
    }

    /* Plant card */
    .dash-plant-card{
        border-radius: 24px;
        padding: 30px 20px 24px 20px;
        display: flex;
        align-items: center;
        text-align: center;
        overflow: hidden !important;
    }

    /* Plant name */
    .dash-plant-name{
        font-size: 28px;
        font-weight: 800;
        letter-spacing: .3px;
        line-height: 1.2;
        margin: 0 0 6px 0;
        color: #1a1a1a;
        text-shadow: 0 2px 4px rgba(0,0,0,0.08);
    }

    /* Plant image container */
    .dash-img-container{
        width: 160px !important;
        height: 140px !important;
        margin: 14px auto 10px auto !important;
        display: flex !important;
        align-items: center !important;
        justify-content: center !important;
        background: rgba(255,255,255,0.95) !important;
        border-radius: 18px !important;
        overflow: hidden !important;
        box-shadow: 0 5px 15px rgba(0,0,0,0.18) !important;
        border: 2px solid rgba(255,255,255,0.45) !important;
    }

    /* Plant image */
    .dash-plant-img{
        max-width: 100% !important;
        max-height: 100% !important;
        width: auto !important;
        height: auto !important;
        object-fit: contain !important;
        display: block !important;
        border-radius: 0 !important;
        box-shadow: none !important;
        border: none !important;
    }

    /* Plant location pill */
    .dash-plant-loc{
        font-size: 15px;
        font-weight: 600;
        background: rgba(255,255,255,0.25);
        padding: 4px 12px;
        border-radius: 20px;
        display: inline-block;
        backdrop-filter: blur(4px);
        margin-top: 2px;
    }

    /* Disease box */
    .dash-disease-box{
        width: 100%;
        background: rgba(255,255,255,0.95);
        color: #333;
        padding: 12px;
        border-radius: 12px;
        margin-top: 16px;
        box-shadow: 0 4px 6px rgba(0,0,0,0.10);
    }

    /* Disease label */
    .dash-disease-label{
        font-size: 12px;
        color: #666;
        display: block;
        margin-bottom: 4px;
        text-transform: uppercase;
        letter-spacing: 1px;
        font-weight: 800;
    }

    /* Disease value */
    .dash-disease-value{
        font-size: 20px;
        font-weight: 900;
        color: #d32f2f;
    }

    /* Healthy disease value */
    .dash-healthy-value{
        font-size: 20px;
        font-weight: 900;
        color: #388e3c;
    }

    /* ===================== SEARCH ENGINE ===================== */

    /* Search panel */
    .se-panel{
        max-width: 980px;
        margin: 10px auto;
        padding: 8px 2px;
        font-family: Inter, system-ui, -apple-system, "Segoe UI", Arial;
    }

    /* Search title */
    .se-title h3{
        margin: 0;
        font-size: 20px;
        color: #4285f4;
        font-weight: 900;
        letter-spacing: .2px;
    }

    /* Search textarea */
    .se-query textarea{
        border-radius: 12px !important;
        border: 1px solid #dfe3eb !important;
        background: linear-gradient(180deg, #f7faff 0%, #ffffff 100%) !important;
        padding: 12px !important;
        outline: none !important;
        box-shadow: none !important;
        font-size: 14px !important;
        line-height: 1.6 !important;
        transition: box-shadow .2s ease, border-color .2s ease, transform .12s ease !important;
        resize: vertical !important;
    }

    /* Search textarea focus */
    .se-query textarea:focus{
        border-color: #1a73e8 !important;
        box-shadow: 0 0 0 4px rgba(26,115,232,0.15) !important;
    }

    /* Search textarea hover */
    .se-query textarea:hover{
        border-color: #c9d7ff !important;
    }

    /* Search actions row */
    .se-actions{
        width: 100% !important;
        gap: 12px !important;
        margin-top: 8px;
    }

    /* Search actions buttons sizing */
    .se-actions .widget-button{
        flex: 1 1 0 !important;
    }

    /* Search actions buttons full width */
    .se-actions .widget-button button{
        width: 100% !important;
    }

    /* Search button base */
    .se-btn button{
        border-radius: 14px !important;
        padding: 10px 18px !important;
        font-weight: 800 !important;
        letter-spacing: .2px !important;
        transition: transform .12s ease, box-shadow .18s ease, background .18s ease, border-color .18s ease !important;
    }

    /* Search button active */
    .se-btn button:active{
        transform: translateY(1px) scale(0.99) !important;
    }

    /* Search primary button */
    .se-btn-primary button{
        background: linear-gradient(180deg, #1a73e8 0%, #155bb8 100%) !important;
        border: 1px solid #155bb8 !important;
        color: #fff !important;
        box-shadow: 0 10px 22px rgba(26,115,232,0.22) !important;
    }

    /* Search primary button hover */
    .se-btn-primary button:hover{
        box-shadow: 0 14px 28px rgba(26,115,232,0.28) !important;
        transform: translateY(-1px) !important;
    }

    /* Search secondary button */
    .se-btn-ghost button{
        background: #ffffff !important;
        border: 1px solid #dfe3eb !important;
        color: #202124 !important;
        box-shadow: 0 8px 18px rgba(16,24,40,0.06) !important;
    }

    /* Search secondary button hover */
    .se-btn-ghost button:hover{
        background: rgba(26,115,232,0.06) !important;
        border-color: #c9d7ff !important;
        box-shadow: 0 12px 22px rgba(16,24,40,0.08) !important;
        transform: translateY(-1px) !important;
    }

    /* Search status text */
    .se-status{
        margin-top: 6px;
        color: #5f6368;
        font-size: 13px;
    }

    /* Search result card */
    .se-results-card{
        margin-bottom: 20px;
        padding: 15px;
        border: 1px solid #e0e0e0;
        border-radius: 8px;
        background: white;
    }
    .image-container{
      background:white;
      border-radius:12px;
      box-shadow:0 4px 12px rgba(0,0,0,0.08);
      width:230px;
      display:flex;
      flex-direction:column;
      overflow:hidden;
      border:1px solid #eee;
    }
    .image{
      position:relative;
      height:140px;
      background:#f4f4f4;
    }
    .image-style{
      width:100%;
      height:100%;
      object-fit:cover;
    }
    .plant-container{
      padding:12px;
      flex-grow:1;
      display:flex;
      flex-direction:column;
    }
    .plant-name{
      margin:0 0 5px 0;
      font-size:16px;
      color:#333;
    }
    .plant-location{
      font-size:12px;
      color:#777;
      margin-bottom:10px;
    }
    </style>
    """))


#Configuration


In [ ]:
#FireBase config
FIREBASE_URL_PLANET_INDEX = "https://purrform-a5da9-default-rtdb.firebaseio.com/plant_disease_index.json"
FIREBASE_DATA_PLANET = "https://purrform-a5da9-default-rtdb.firebaseio.com/plant_disease_logs.json"
preds = None


# Sensors API
BASE_URL = "https://server-cloud-v645.onrender.com/"

# Articles for search engine and index
DOCUMENTS = {
    1: {
        'title': 'Advanced methods of plant disease detection',
        'url': 'https://link.springer.com/article/10.1007/s13593-014-0246-1',
        'domain': 'Agricultural sustainability',
        'summary': 'Comprehensive strategies for sustainable crop protection focusing on integrated pest management and ecological approaches.'
    },
    2: {
        'title': 'Rice Plant Disease Detection and Classification Using Deep Residual Learning',
        'url': 'https://link.springer.com/chapter/10.1007/978-981-15-6315-7_23',
        'domain': 'Computer science applications',
        'summary': 'Advanced machine learning techniques for automated plant disease detection and classification using image analysis.'
    },
    3: {
        'title': 'Development Status and Trend of IoT Communication Technology',
        'url': 'https://link.springer.com/chapter/10.1007/978-981-15-5959-4_11',
        'domain': 'Plant health technology',
        'summary': 'Modern monitoring technologies including IoT sensors and remote sensing for real-time plant health assessment.'
    },
    4: {
        'title': 'Distributed Optical Fiber Sensing System Performance Improvement Using Signal Processing Techniques',
        'url': 'https://link.springer.com/chapter/10.1007/978-981-15-2774-6_5',
        'domain': 'Plant pathology',
        'summary': 'Comprehensive analysis of fungal diseases affecting major agricultural crops and management strategies.'
    },
    5: {
        'title': 'Flood Detection Using Multispectral Images and SAR Data',
        'url': 'https://link.springer.com/chapter/10.1007/978-981-15-6315-7_24',
        'domain': 'Precision agriculture',
        'summary': 'Integration of precision agriculture technologies for targeted disease management and prevention strategies.'
    }
}

#upload photo form

In [ ]:
"""
Plant image scanning (disease classification) + Firebase RTDB logging + Upload UI (ipywidgets).

This module:
- Runs a HuggingFace image-classification pipeline on an uploaded plant photo
- Extracts (plant, disease) from the model label
- Stores the best prediction + metadata in Firebase Realtime Database (RTDB)
- Updates UI messages and refreshes dashboard/history tabs
- Switches to the Dashboard tab automatically after a successful save
"""

_session = requests.Session()

clf = pipeline(
    "image-classification",
    model="linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"
)

def predict(image):
    """Run the classifier on a PIL image and return the top-5 labels mapped to float scores."""
    preds = clf(image)
    return {p["label"]: float(p["score"]) for p in preds[:5]}

def normalize_label(s: str) -> str:
    """Normalize a raw prediction label by replacing underscores and collapsing repeated whitespace."""
    return re.sub(r"\s+", " ", s.replace("_", " ")).strip()

def clean_plant_name(plant: str) -> str:
    """Clean a plant name by removing a trailing 'plant' token (if present) and trimming whitespace."""
    return re.sub(r"\bplant\b$", "", plant.strip(), flags=re.IGNORECASE).strip()

def extract_plant_disease(label: str):
    """Parse a model label into (plant_name, disease_name) using multiple known label patterns."""
    s = normalize_label(label)
    low = s.lower()

    if low.startswith("healthy "):
        plant = clean_plant_name(s[len("Healthy "):].strip())
        return plant, "Healthy"

    if ("cedar" in low) and ("apple" in low) and ("rust" in low):
        return "Apple", "Cedar Apple Rust"

    if "___" in s:
        plant, disease = s.split("___", 1)
        return clean_plant_name(plant.strip()), disease.strip()

    if " with " in s:
        plant, disease = s.split(" with ", 1)
        return clean_plant_name(plant.strip()), disease.strip()

    parts = s.split()
    if len(parts) >= 2:
        plant = clean_plant_name(" ".join(parts[:-1]))
        disease = parts[-1]
        return plant, disease

    return s, s

warnings.filterwarnings(
    "ignore",
    message=r"Palette images with Transparency expressed in bytes should be converted to RGBA images"
)

def open_image_safely(content_bytes: bytes) -> Image.Image:
    """Open raw image bytes and convert to RGB, safely handling palette images with transparency."""
    img = Image.open(io.BytesIO(content_bytes))
    if img.mode == "P" and ("transparency" in img.info):
        img = img.convert("RGBA").convert("RGB")
    else:
        img = img.convert("RGB")
    return img

def image_bytes_to_data_url(content_bytes: bytes, max_side: int = 500, jpeg_quality: int = 70) -> str:
    """Convert image bytes to a resized JPEG data-URL for lightweight UI display and storage."""
    img = open_image_safely(content_bytes)

    w, h = img.size
    scale = min(max_side / max(w, h), 1.0)
    if scale < 1.0:
        img = img.resize((int(w * scale), int(h * scale)))

    buf = io.BytesIO()
    img.save(buf, format="JPEG", quality=jpeg_quality, optimize=True)
    jpeg_bytes = buf.getvalue()

    b64 = base64.b64encode(jpeg_bytes).decode("utf-8")
    return f"data:image/jpeg;base64,{b64}"

def save_max_to_rtdb(plant, disease, score, address, file_name, image_url):
    """Write a single scan result (best prediction + metadata) into Firebase RTDB and return the RTDB response."""
    # Requires FIREBASE_DB_URL, FIREBASE_NODE, FIREBASE_AUTH to be defined in your notebook.
    data = {
        "plant_name": plant,
        "disease": disease,
        "address": address,
        "image_url": image_url,
        "timestamp": datetime.now(timezone.utc).isoformat()
    }

    url = f"{FIREBASE_DB_URL.rstrip('/')}/{FIREBASE_NODE}.json"
    params = {}
    if FIREBASE_AUTH:
        params["auth"] = FIREBASE_AUTH

    resp = _session.post(url, params=params, json=data, timeout=20)
    if resp.status_code not in (200, 201):
        raise RuntimeError(f"RTDB write failed: {resp.status_code} | {resp.text[:250]}")
    return resp.json() if resp.text else None

def create_plant_upload_form():
    """Create the upload form UI: select an image, input an address, run inference, save to RTDB, refresh, and switch tab."""
    title = widgets.HTML("""
    <div class="plant-form-title">Upload photo</div>
    <div class="plant-form-sub">
        Upload a plant image and fill in the details to start AI analysis.
    </div>
    """)

    upload = widgets.FileUpload(accept='image/*', multiple=False, description='Upload photo')
    upload_box = widgets.Box([upload])
    upload_box.add_class("plant-upload-box")

    plant_address_label = widgets.HTML('<div class="plant-label">Address (required):</div>')
    plant_address_input = widgets.Text(
        placeholder='Field location (required)',
        layout=widgets.Layout(width='99%')
    )
    plant_address_box = widgets.VBox([plant_address_label, plant_address_input])
    plant_address_box.add_class("plant-input")

    start_btn = widgets.Button(description='Start Analysis')
    start_btn_box = widgets.HBox([start_btn])
    start_btn_box.add_class("plant-start-btn")

    message_html = widgets.HTML("")
    message_box = widgets.VBox([message_html])
    message_box.add_class("plant-message")

    result_out = widgets.Output()

    def on_start_click(_):
        """Validate inputs, run prediction, save to RTDB, refresh other tabs, then switch to the Dashboard tab."""
        message_html.value = ""
        with result_out:
            clear_output(wait=True)

        if not upload.value:
            message_html.value = """<div class="plant-message-error plant-message">⚠️ Please upload a plant photo first.</div>"""
            return

        if plant_address_input.value.strip() == "":
            message_html.value = """<div class="plant-message-error plant-message">⚠️ Please enter the address (required).</div>"""
            return

        file_info = list(upload.value.values())[0]
        file_name = file_info["metadata"]["name"]
        content = file_info["content"]

        image = open_image_safely(content)

        message_html.value = f"""
        <div class="plant-message-success plant-message">
            🔄 Analyzing (<b>{file_name}</b>) — Address: <b>{plant_address_input.value.strip()}</b>
        </div>
        """

        with result_out:
            preds = predict(image)
            best_label, best_score = max(preds.items(), key=lambda x: x[1])
            best_plant, best_disease = extract_plant_disease(best_label)

            image_url = image_bytes_to_data_url(content, max_side=250, jpeg_quality=70)

            try:
                save_max_to_rtdb(
                    plant=best_plant,
                    disease=best_disease,
                    score=best_score,
                    address=plant_address_input.value.strip(),
                    file_name=file_name,
                    image_url=image_url
                )

                message_html.value = f"""
                <div class="plant-message-success plant-message">
                    ✅ The file <b>{file_name}</b> was analyzed successfully: <b>{best_plant}</b> diagnosed as <b>{best_disease}</b>.
                </div>
                """

                try:
                    refresh_dashboard_tab()
                except Exception:
                    pass

                try:
                    refresh_ai_tab()
                except Exception:
                    pass

                # Auto-switch to the Dashboard tab (index 3)
                try:
                    tabs.selected_index = 3
                except Exception:
                    pass

            except Exception as e:
                message_html.value = f"""
                <div class="plant-message-error plant-message">
                    ⛔ Save failed: {str(e)[:220]}
                </div>
                """

    start_btn.on_click(on_start_click)

    uploadFormUI = widgets.VBox([
        title,
        upload_box,
        plant_address_box,
        start_btn_box,
        message_box,
        result_out
    ])
    uploadFormUI.add_class("plant-form")
    return uploadFormUI

# Output container for Tab 1
tab1_upload = widgets.Output()
with tab1_upload:
    clear_output(wait=True)
    display(create_plant_upload_form())


#IoT Sensors form

In [ ]:
"""
IoT Sensors UI (ipywidgets) + live history fetch from a REST API.

This module:
- Pulls the last 10 sensor samples (temperature/humidity/soil) from BASE_URL
- Keeps a short history list for charts + latest values for gauges
- Renders a dashboard-like IoT tab with a refresh button, gauges, and graphs
"""



# Global variables for sensor data storage
sensor_history = {
    'temperature': [],
    'humidity': [],
    'soil': []
}

# Current sensor data (default values)
sensor_data = {
    'temperature': 27,
    'humidity': 60,
    'soil': 55
}

def fetch_sensor_data():
    """Fetch the latest 10 samples for all sensors from the API and update sensor_history + sensor_data."""
    global sensor_history, sensor_data

    try:
        # Request last N samples (API returns a list under "data")
        response = requests.get(f"{BASE_URL}/history", params={"feed": "json", "limit": 10})
        data = response.json()

        if "data" in data:
            # Reset history before loading fresh samples
            sensor_history = {'temperature': [], 'humidity': [], 'soil': []}

            # Parse each sample into per-metric history arrays
            for sample in data["data"]:
                try:
                    # Each sample["value"] is a JSON string holding all sensor metrics
                    sensor_reading = json.loads(sample["value"])

                    if "temperature" in sensor_reading:
                        sensor_history['temperature'].append(float(sensor_reading["temperature"]))
                    if "humidity" in sensor_reading:
                        sensor_history['humidity'].append(float(sensor_reading["humidity"]))
                    if "soil" in sensor_reading:
                        sensor_history['soil'].append(float(sensor_reading["soil"]))

                except (json.JSONDecodeError, ValueError, KeyError):
                    # Skip malformed samples rather than failing the whole refresh
                    continue

            # Update current values to the most recent sample in each list (if present)
            if sensor_history['temperature']:
                sensor_data['temperature'] = sensor_history['temperature'][-1]
            if sensor_history['humidity']:
                sensor_data['humidity'] = sensor_history['humidity'][-1]
            if sensor_history['soil']:
                sensor_data['soil'] = sensor_history['soil'][-1]

            return True

    except Exception as e:
        print(f"Error fetching data: {e}")
        return False

    return False

def _get_status_class(metric_type, value, percentage):
    """Return the CSS status class (optimal/warning/critical) based on the metric thresholds."""
    if metric_type == "temperature":
        if value < 10 or value > 35:
            return "status-critical"
        elif value < 15 or value > 28:
            return "status-warning"
        return "status-optimal"

    if metric_type == "humidity":
        if percentage < 30:
            return "status-critical"
        elif percentage < 50:
            return "status-warning"
        return "status-optimal"

    if metric_type == "soil_moisture":
        if percentage < 20 or percentage > 80:
            return "status-critical"
        elif percentage < 40 or percentage > 70:
            return "status-warning"
        return "status-optimal"

def create_circular_gauge(value, max_value, min_value=0, unit="", icon="📊",
                         metric_name="", metric_type=""):
    """Build a circular gauge representing the metric value as a progress ring."""
    percentage = ((value - min_value) / (max_value - min_value)) * 100
    circumference = 2 * 3.14159 * 45
    stroke_dashoffset = circumference - (percentage / 100) * circumference

    status_class = _get_status_class(metric_type, value, percentage)

    # Widget: HTML gauge (SVG + value + label)
    html_content = f"""
    <div class="metric-item">
        <div class="metric-icon {status_class}">{icon}</div>
        <div class="circular-progress">
            <svg class="progress-ring" width="110" height="110">
                <circle class="progress-ring__circle bg-circle"
                        stroke-width="6"
                        fill="transparent"
                        r="45"
                        cx="55"
                        cy="55"/>
                <circle class="progress-ring__circle {status_class}"
                        stroke-width="6"
                        fill="transparent"
                        r="45"
                        cx="55"
                        cy="55"
                        stroke-linecap="round"
                        stroke-dasharray="{circumference}"
                        stroke-dashoffset="{stroke_dashoffset}"/>
            </svg>
            <div class="metric-value">{value}{unit}</div>
        </div>
        <div class="metric-label">{metric_name}</div>
    </div>
    """

    return widgets.HTML(html_content)

def create_sensor_card(sensor_name, soil_moisture, humidity, temperature):
    """Create a sensor card with three gauges: soil moisture, humidity, and temperature."""
    # Widget: gauge (soil moisture)
    soil_gauge = create_circular_gauge(
        value=soil_moisture, max_value=100, min_value=0, unit="%",
        icon="🌱", metric_name="Soil Moisture", metric_type="soil_moisture"
    )

    # Widget: gauge (humidity)
    humidity_gauge = create_circular_gauge(
        value=humidity, max_value=100, min_value=0, unit="%",
        icon="💧", metric_name="Humidity", metric_type="humidity"
    )

    # Widget: gauge (temperature)
    temperature_gauge = create_circular_gauge(
        value=temperature, max_value=50, min_value=0, unit="°C",
        icon="🌡️", metric_name="Temperature", metric_type="temperature"
    )

    # Widget: sensor title/header (HTML)
    sensor_header = widgets.HTML(f'<div class="sensor-name">{sensor_name}</div>')

    # Widget: metrics row (HBox) holding all three gauges
    metrics_container = widgets.HBox([soil_gauge, humidity_gauge, temperature_gauge])
    metrics_container.add_class("metrics-container")

    # Widget: card container (VBox)
    card = widgets.VBox([sensor_header, metrics_container])
    card.add_class("sensor-card")
    return card

def create_iot_sensors_tab():
    """Build the IoT Sensors tab UI (refresh button + sensor card + last-10-samples graphs)."""

    # Widget: refresh button (click triggers data fetch + UI re-render)
    refresh_btn = widgets.Button(
        description='🔄 Refresh Data',
        button_style='',
        layout=widgets.Layout(width='auto')
    )

    # Widget: refresh button wrapper (HBox) for centering + CSS styling
    refresh_container = widgets.HBox(
        [refresh_btn],
        layout=widgets.Layout(justify_content='center', width='100%')
    )
    refresh_container.add_class("refresh-btn-container")

    # Widget: header/title (HTML)
    header_html = """
    <div class="iot-header">
        <div class="iot-title">IoT Sensors</div>
    </div>
    """
    header = widgets.HTML(header_html)

    # Widget: output area for sensor card (cleared and re-rendered on refresh)
    sensor_content = widgets.Output()

    # Widget: output area for graphs (cleared and re-rendered on refresh)
    graph_content = widgets.Output()

    def refresh_data(b=None):
        """Fetch latest API data and update both the sensor card + the graphs outputs."""
        # Clear outputs first to avoid stacking multiple renders
        sensor_content.clear_output(wait=True)
        graph_content.clear_output(wait=True)

        success = fetch_sensor_data()

        if success:
            # Render the sensor card (latest values)
            with sensor_content:
                # Widget: card for a single sensor station
                sensor1 = create_sensor_card(
                    "Sensor 1",
                    sensor_data['soil'],
                    sensor_data['humidity'],
                    sensor_data['temperature']
                )
                display(sensor1)

            # Render graphs (history lists)
            with graph_content:
                # Widget: graph container (VBox) with a title
                graph_container = widgets.VBox([
                    widgets.HTML('<div class="graph-title">Last 10 samples</div>')
                ])
                graph_container.add_class("graph-container")

                if any(sensor_history.values()):
                    # Widget: dedicated output per graph to isolate plt.show() rendering
                    soil_container = widgets.Output()
                    soil_container.add_class("single-graph-container")

                    humidity_container = widgets.Output()
                    humidity_container.add_class("single-graph-container")

                    temp_container = widgets.Output()
                    temp_container.add_class("single-graph-container")

                    with soil_container:
                        create_single_graph('soil', sensor_history['soil'], '%', '#00d084', '#f0fff4')

                    with humidity_container:
                        create_single_graph('humidity', sensor_history['humidity'], '%', '#4285f4', '#f0f7ff')

                    with temp_container:
                        create_single_graph('temperature', sensor_history['temperature'], '°C', '#ff6b6b', '#fff5f5')

                    # Widget: row layout for the 3 graphs
                    graphs_row = widgets.HBox([soil_container, humidity_container, temp_container])
                    graphs_row.add_class("graphs-container")

                    # Inject the graphs row into the container
                    graph_container.children = list(graph_container.children) + [graphs_row]
                else:
                    # Widget: empty-state message (HTML)
                    empty_msg = widgets.HTML(
                        '<div style="text-align: center; color: white;">No historical data available for graphs</div>'
                    )
                    graph_container.children = list(graph_container.children) + [empty_msg]

                display(graph_container)

        else:
            with sensor_content:
                # Widget: fallback card using default values if the server is unavailable
                sensor1 = create_sensor_card(
                    "Sensor 1",
                    sensor_data['soil'],
                    sensor_data['humidity'],
                    sensor_data['temperature']
                )
                display(sensor1)
                print("Using default values (server unavailable)")

    # Wire the refresh button to refresh_data()
    refresh_btn.on_click(refresh_data)

    # Initial render (so the tab shows content immediately)
    refresh_data()

    # Widget: main tab container (VBox)
    iot_interface = widgets.VBox([
        header,
        refresh_container,
        sensor_content,
        graph_content
    ])
    iot_interface.add_class("iot-container")
    return iot_interface

def create_single_graph(sensor_type, data, unit, color, bg_color):
    """Plot one metric history series (last N samples) using Matplotlib."""
    if not data:
        print(f"No {sensor_type} data available")
        return

    fig, ax = plt.subplots(1, 1, figsize=(5, 4))
    fig.patch.set_facecolor('white')

    x_vals = list(range(1, len(data) + 1))
    ax.plot(
        x_vals, data,
        color=color, linewidth=3, marker='o',
        markersize=6, markerfacecolor=color,
        markeredgecolor='white', markeredgewidth=2
    )

    ax.set_facecolor(bg_color)
    ax.set_title(sensor_type.capitalize(), fontsize=14, fontweight='bold', color='#2c3e50')
    ax.grid(True, alpha=0.3, color=color)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color(color)
    ax.spines['bottom'].set_color(color)

    ax.set_ylabel(unit, fontsize=11, color='#5f6368')
    ax.set_xlabel('Sample', fontsize=11, color='#5f6368')
    ax.tick_params(colors='#5f6368', labelsize=9)

    # Keep axis limits consistent across refreshes for better visual comparison
    if sensor_type == 'temperature':
        ax.set_ylim(0, 50)
    else:
        ax.set_ylim(0, 100)

    plt.tight_layout()
    plt.show()

# Initial setup
fetch_sensor_data()
print("IoT Sensor interface ready with new API integration")


# Analysis history

In [ ]:
tab3_ai = widgets.Output()

# Widget: tab header (HTML)
header_html = """
<div class="iot-header">
    <div class="iot-title">Analysis history</div>
</div>
"""
# Widget: header element (HTML)
header = widgets.HTML(header_html)

def fetch_all_history():
    """Fetch all scans from Firebase RTDB and return them sorted by newest timestamp first."""
    url = f"{FIREBASE_DATA_PLANET}"
    params = {}
    if FIREBASE_AUTH:
        params["auth"] = FIREBASE_AUTH

    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()
    data = r.json() or {}

    items = []
    # convert RTDB dict-of-records into a list and inject the record id
    for key, val in data.items():
        if isinstance(val, dict):
            val = dict(val)
            val["id"] = key
            items.append(val)

    items.sort(key=lambda x: x.get("timestamp", ""), reverse=True)
    return items

def create_history_card(item):
    """Build a single HTML card for one scan record (image, plant name, location, disease status)."""
    p_name    = item.get("plant_name", "Unknown")
    p_img     = item.get("image_url") or "https://via.placeholder.com/150?text=No+Image"
    p_loc     = item.get("address", "Unknown Location")
    p_disease = item.get("disease", "Healthy")
    p_score   = item.get("score", 0)

    # Disease status normalization: treat any 'healthy' (case/space-insensitive) as a healthy result
    is_healthy = str(p_disease).strip().lower() == "healthy"

    # Card theme colors: green for healthy, red for non-healthy
    status_color = "#00c853" if is_healthy else "#ff3d00"
    bg_color     = "#f1f8e9" if is_healthy else "#ffebee"

    # Display label: show 'Healthy' or the raw disease string
    disease_display = "Healthy" if is_healthy else f"{p_disease}"

    return f"""
    <div class="image-container">
        <div class="image">
            <img src="{p_img}" class="image-style"
                 onerror="this.src='https://via.placeholder.com/150?text=Error'">
        </div>

        <div class="plant-container">
            <h3 class="plant-name">{p_name}</h3>
            <div style="plant-location">📍 {p_loc}</div>

            <div style="margin-top:auto;background:{bg_color};padding:6px;border-radius:6px;text-align:center;
                        border:1px solid {status_color}30;">
                <div style="font-size:13px;font-weight:bold;color:{status_color};">{disease_display}</div>
            </div>
        </div>
    </div>
    """

def render_ai_analysis_tab():
    """Render the full Analysis History tab (header + responsive grid of scan cards)."""
    try:
        history_items = fetch_all_history()
    except Exception as e:
        return HTML(f"<div style='padding:20px;color:#b00;'>Error fetching history: {str(e)[:200]}</div>")

    if not history_items:
        return HTML("<div style='padding:20px;color:#666;'>No scans found.</div>")

    cards_html = "".join(create_history_card(item) for item in history_items)
    full_html = f"""
    <div style="font-family:'Segoe UI',sans-serif;padding:10px;">
        {header_html}
        <div style="display:flex;flex-wrap:wrap;gap:20px;margin-top:12px;">
            {cards_html}
        </div>
    </div>
    """
    return HTML(full_html)

def refresh_ai_tab():
    """Refresh the tab output by clearing and re-displaying the latest rendered HTML."""
    with tab3_ai:
        clear_output(wait=True)
        display(render_ai_analysis_tab())

refresh_ai_tab()


#Dashboard


In [ ]:
FIREBASE_DB_URL = "https://purrform-a5da9-default-rtdb.firebaseio.com"
FIREBASE_NODE   = "plant_disease_logs"
FIREBASE_AUTH   = None

_session = requests.Session()

def fetch_weather_open_meteo(lat=32.917, lon=35.300, location_name="North Districts, Israel"):
    """Fetch current temperature and a 7-day max-temperature forecast from Open-Meteo for the given coordinates."""
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "current": "temperature_2m",
        "daily": "temperature_2m_max",
        "timezone": "auto",
        "forecast_days": 7,
    }

    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    j = r.json()

    temp_today = int(round(j["current"]["temperature_2m"]))

    week_forecast = []
    # Build the 7-day forecast list as compact day + rounded temperature pairs
    for t, temp in zip(j["daily"]["time"], j["daily"]["temperature_2m_max"]):
        d = date.fromisoformat(t)
        week_forecast.append({"day": d.strftime("%a"), "temp": int(round(temp))})

    return {
      "date": datetime.now(ZoneInfo("Asia/Jerusalem")).strftime("%d/%m/%Y"),
      "location": location_name,
      "temp_today": temp_today,
      "week_forecast": week_forecast,
    }

# =========================
# RTDB Fetch last plant
# =========================
def fetch_last_plant_from_rtdb(db_url, node, auth=None):
    """Fetch the most recent plant scan entry from Firebase RTDB by parsing and sorting ISO timestamps."""
    url = f"{db_url.rstrip('/')}/{node}.json"
    params = {}
    if auth:
        params["auth"] = auth

    r = _session.get(url, params=params, timeout=20)
    r.raise_for_status()

    data = r.json() or {}
    if not data:
        return None

    valid = []
    # Filter RTDB records and keep only entries that have a parseable ISO timestamp
    for _, v in data.items():
        if not isinstance(v, dict):
            continue

        ts = v.get("timestamp")
        if not ts:
            continue

        try:
            dt = datetime.fromisoformat(ts.replace("Z", "+00:00"))
        except Exception:
            continue

        valid.append((dt, v))

    if not valid:
        return None

    # Sort by timestamp and return the newest record
    valid.sort(key=lambda x: x[0])
    return valid[-1][1]

def build_plant_card_from_db(db_item):
    """Convert an RTDB record into the normalized 'plant_card' dict used by the dashboard UI."""
    if not db_item:
        return {
            "name": "Unknown",
            "image": "",
            "location": "Unknown",
            "stats": {"diseases": {"count": 0, "type": None}}
        }

    disease = db_item.get("disease")
    plant_name = db_item.get("plant_name", "Unknown")
    image_url = db_item.get("image_url", "")
    address = db_item.get("address", "Unknown")

    # Normalize disease to a boolean flag: treat empty/whitespace or "healthy" (case-insensitive) as no disease
    has_disease = 1 if (disease and str(disease).strip() and str(disease).lower() != "healthy") else 0

    return {
        "name": plant_name,
        "image": image_url,
        "location": address,
        "stats": {
            "diseases": {"count": has_disease, "type": disease if has_disease else None}
        }
    }

# =========================
# Data Builder
# =========================
PLANTS = ["Tomato", "Banana", "Cucumber", "Pepper"]

def create_fake_dashboard_data(use_api=True):
    """Build the dashboard data model (weather + last scanned plant); fallback to random weather if API fails."""
    weather = None

    # Try real weather API when enabled, otherwise fallback to local random values
    if use_api:
        try:
            weather = fetch_weather_open_meteo()
        except Exception:
            weather = None

    # Fallback weather payload uses the same shape as the API response for consistent rendering
    if weather is None:
        week_days = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
        weather = {
            "date": date.today().strftime("%d/%m/%Y"),
            "location": "North Districts, Israel",
            "temp_today": random.randint(18, 33),
            "week_forecast": [{"day": d, "temp": random.randint(18, 33)} for d in week_days],
        }

    # Attach the most recent plant scan into the same data object under "plant_card"
    try:
        last_item = fetch_last_plant_from_rtdb(FIREBASE_DB_URL, FIREBASE_NODE, auth=FIREBASE_AUTH)
        weather["plant_card"] = build_plant_card_from_db(last_item)
    except Exception:
        weather["plant_card"] = build_plant_card_from_db(None)

    return weather

# =========================
# UI Components (Dashboard)
# =========================
def create_dashboard_plant_card(plant):
    """Create the right-side dashboard card that displays the last scanned plant and its diagnosis."""
    name = plant.get("name", "Unknown")
    img_url = plant.get("image", "")
    location = plant.get("location", "Unknown Location")

    diseases = (plant.get("stats", {}).get("diseases", {}) or {})
    has_disease = diseases.get("count", 0) > 0
    disease_type = diseases.get("type") if has_disease else None

    # Widget: plant title
    title = widgets.HTML(f"<div class='dash-plant-name'>{name}</div>")

    # Widget: plant image container (keeps the image clipped + centered)
    img = widgets.HTML(f"""
        <div style="
            width: 220px; height: 160px;
            margin: 10px auto 6px auto;
            border-radius: 14px;
            overflow: hidden;
            background: white;
            box-shadow: 0 10px 25px rgba(0,0,0,0.12);
            display:flex; align-items:center; justify-content:center;">
            <img src="{img_url}" alt="{name}" style="width:100%;height:100%;object-fit:cover;display:block;">
        </div>
    """)

    # Widget: location badge
    loc = widgets.HTML(f"""<span class="dash-plant-loc">📍 {location}</span>""")

    # Widget: diagnosis label + value (disease vs healthy style)
    diag_label = widgets.HTML("""<span class="dash-disease-label">Diagnosis</span>""")
    diag_value = widgets.HTML(
        f"""<span class="dash-disease-value">{disease_type or "Unknown"}</span>"""
        if has_disease
        else """<span class="dash-healthy-value">Healthy</span>"""
    )

    # Widget: diagnosis box wrapper
    disease_box = widgets.VBox([diag_label, diag_value])
    disease_box.add_class("dash-disease-box")

    # Widget: full plant card
    card = widgets.VBox([title, img, loc, disease_box])
    card.add_class("dash-card")
    card.add_class("dash-plant-card")
    return card

def create_dashboard_tab(data=None):
    """Render the full Dashboard tab: weather card (left) + last plant card (right)."""
    # Data source: build a fresh model when no external data is provided
    if data is None:
        data = create_fake_dashboard_data()

    # Widget: dashboard title
    title = widgets.Label("Dashboard")
    title.add_class("dash-title")

    # Widget: weather card title
    w_title = widgets.HTML("☀️ <b>Today's Weather</b>")
    w_title.add_class("dash-card-title")

    # Widget: today's temperature display
    w_temp = widgets.HTML(f"""
    <div style="display:flex; align-items:center; justify-content:center; gap:10px;">
        <span style="font-size:28px;">🌤️</span>
        <span class="dash-weather-temp" style="font-size:44px; font-weight:800;">{data['temp_today']}°</span>
    </div>
    """)

    # Widget: weather location line
    w_loc = widgets.HTML(f"📍 <span style='font-size:15px; font-weight:600;'>{data['location']}</span>")
    w_loc.add_class("dash-subtext")

    # Widget: date line
    w_date = widgets.HTML(f"📅 <span style='font-size:15px; font-weight:600;'>{data['date']}</span>")
    w_date.add_class("dash-subtext")

    # Widget: location + date group
    meta_box = widgets.VBox([w_loc, w_date], layout=widgets.Layout(align_items="center", margin="18px 0 0 0"))

    forecast_items = []
    # Build forecast tiles so each day has identical widget structure and styles
    for item in data["week_forecast"]:
        day_lbl = widgets.Label(item["day"]); day_lbl.add_class("dash-forecast-day")
        temp_lbl = widgets.Label(f"{item['temp']}°"); temp_lbl.add_class("dash-forecast-temp")

        day_box = widgets.VBox(
            [day_lbl, temp_lbl],
            layout=widgets.Layout(
                flex="1 1 0", min_width="0px", height="130px",
                align_items="center", justify_content="center", padding="10px 0 0 0"
            )
        )
        day_box.add_class("dash-forecast-box")
        forecast_items.append(day_box)

    # Widget: forecast row (one HBox containing all day tiles)
    forecast_row = widgets.HBox(
        forecast_items,
        layout=widgets.Layout(width="100%", height="170px", margin="80px 0 0 0",
                              justify_content="space-between", align_items="stretch")
    )
    forecast_row.add_class("dash-weather-week")

    # Widget: weather content layout inside the weather card
    weather_content = widgets.VBox(
        [w_title, w_temp, meta_box, forecast_row],
        layout=widgets.Layout(width="100%", align_items="center", justify_content="flex-start",
                              gap="8px", padding="24px 0 0 0")
    )

    # Widget: full weather card
    weather_card = widgets.VBox([weather_content])
    weather_card.add_class("dash-card")
    weather_card.add_class("dash-card-weather")

    # Widget: plant card (last scan from RTDB)
    plant_card = create_dashboard_plant_card(data.get("plant_card", {}))

    # Layout: enforce equal sizing so both cards align nicely side-by-side
    card_layout = widgets.Layout(flex="1 1 0", min_height="420px", height="520px", margin="0 12px 12px 0")
    weather_card.layout = card_layout
    plant_card.layout = card_layout

    # Widget: main row holding both cards
    row = widgets.HBox(
        [weather_card, plant_card],
        layout=widgets.Layout(width="100%", align_items="stretch", justify_content="space-between")
    )

    # Widget: entire dashboard container
    dashboard_ui = widgets.VBox([title, row], layout=widgets.Layout(width="100%"))
    dashboard_ui.add_class("dash-container")
    return dashboard_ui

# =========================
# Dashboard OUTPUT + refresh function
# =========================
tab4_dashboard = widgets.Output()

def refresh_dashboard_tab():
    """Clear and re-render the Dashboard output so it reflects the latest RTDB + weather data."""
    with tab4_dashboard:
        clear_output(wait=True)
        display(create_dashboard_tab())

refresh_dashboard_tab()


# Index Loader and RAG


In [ ]:
def get_stop_words():
    """Return a compact set of stop-words used to filter common and non-informative query terms."""
    return {
        # Basic English (30 words)
        "the", "and", "for", "with", "that", "this", "are", "was", "were", "from",
        "have", "has", "had", "but", "not", "can", "may", "will", "would", "could",
        "they", "them", "their", "these", "those", "such", "very", "much", "many", "some",

        # Academic/Research terms (15 words)
        "study", "paper", "research", "method", "methods", "result", "results", "analysis",
        "show", "use", "used", "based", "data", "system", "approaches",

        # Generic descriptors (10 words)
        "new", "different", "important", "high", "low", "large", "small", "good", "better", "modern"
    }

def load_firebase_index():
    """Load the inverted index JSON from Firebase RTDB; return None if the request fails or is not OK."""
    try:
        response = requests.get(FIREBASE_URL_PLANET_INDEX, timeout=10)
        if response.status_code == 200:
            return response.json()
    except Exception:
        pass
    return None

def process_query(query):
    """Tokenize a query, remove stop-words, and apply stemming to produce normalized search terms."""
    if not query:
        return []

    words = re.findall(r'\b[a-zA-Z]{3,}\b', query.lower())
    stemmer = PorterStemmer()
    processed_words = []
    stop_words = get_stop_words()

    # Filter words using stop-words, then stem each term for more robust matching
    for word in words:
        if word not in stop_words:
            stemmed = stemmer.stem(word)
            if len(stemmed) >= 3:
                processed_words.append(stemmed)
            else:
                processed_words.append(word)

    return processed_words

def search_documents(query, max_results=5):
    """Search documents using a Firebase-hosted inverted index and return top results sorted by relevance."""
    INDEX = load_firebase_index()
    if not INDEX:
        return []

    query_terms = process_query(query)
    if not query_terms:
        return []

    doc_scores = defaultdict(lambda: {'matches': 0, 'frequency': 0, 'terms': []})

    # Score documents by term matches and term frequencies across documents
    for term in query_terms:
        if term in INDEX:
            term_data = INDEX[term]
            docs_data = term_data.get('DocsIds', {})

            # Index shape A: dict mapping doc_id -> frequency
            if isinstance(docs_data, dict):
                for doc_id_str, frequency in docs_data.items():
                    if doc_id_str and frequency:
                        doc_id = int(doc_id_str)
                        doc_scores[doc_id]['matches'] += 1
                        doc_scores[doc_id]['frequency'] += frequency
                        doc_scores[doc_id]['terms'].append(term)

            # Index shape B: list where position is doc_id and value is frequency
            elif isinstance(docs_data, list):
                for doc_id, frequency in enumerate(docs_data):
                    if frequency is not None and frequency > 0:
                        doc_scores[doc_id]['matches'] += 1
                        doc_scores[doc_id]['frequency'] += frequency
                        doc_scores[doc_id]['terms'].append(term)

    if not doc_scores:
        return []

    results = []
    # Convert raw doc scores into user-facing results by joining with DOCUMENTS metadata
    for doc_id, scores in doc_scores.items():
        doc_info = DOCUMENTS.get(doc_id, {})
        if doc_info:
            relevance_score = scores['matches'] * 100 + scores['frequency']
            results.append({
                'title': doc_info['title'],
                'domain': doc_info['domain'],
                'summary': doc_info['summary'],
                'url': doc_info['url'],
                'score': relevance_score
            })

    results.sort(key=lambda x: x['score'], reverse=True)
    return results[:max_results]


#Search engine

In [ ]:
def build_plant_disease_search_widget():
    """Build a styled search UI that takes a text query, calls search_documents(), and renders ranked paper cards."""
    # Widget: title header (HTML)
    title = widgets.HTML("<div class='se-title'><h3>🌱 Plant Disease Research Engine</h3></div>")

    # Widget: multi-line query input (Textarea)
    query = widgets.Textarea(
        placeholder="Enter your plant disease research query...",
        layout=widgets.Layout(width="100%", height="120px")
    )

    # Widget: search button (Button)
    btn_search = widgets.Button(description="Search", icon="search", button_style="primary")

    # Widget: clear button (Button)
    btn_clear  = widgets.Button(description="Clear",  icon="times",  button_style="")

    # Layout: make each button occupy half of the actions row
    btn_search.layout = widgets.Layout(width="auto", flex="1 1 0")
    btn_clear.layout  = widgets.Layout(width="auto", flex="1 1 0")

    # Styling: attach CSS classes for consistent theme
    btn_search.add_class("se-btn")
    btn_search.add_class("se-btn-primary")
    btn_clear.add_class("se-btn")
    btn_clear.add_class("se-btn-ghost")

    # Widget: status line (HTML)
    status = widgets.HTML("<div class='se-status'>🔍 Ready to search.</div>")

    # Widget: results output area (Output)
    out_results = widgets.Output()

    def on_search(_):
        """Handle Search click: validate input, run search_documents(), and render results into the output panel."""
        # Input validation: ignore empty/whitespace-only queries
        q = query.value.strip()
        if not q:
            status.value = "<div class='se-status' style='color:#b00'>⚠️ Please enter a search query.</div>"
            return

        # UI state: show searching status immediately
        status.value = "<div class='se-status' style='color:#2f63ff'>🔄 Searching...</div>"

        # Search: call the notebook's search_documents() function
        results = search_documents(q, max_results=5)

        with out_results:
            clear_output(wait=True)

            # Render: empty results block + status update
            if not results:
                display(HTML(f"""
                <div style="margin-top:10px; padding:20px; background:#f8f9fa; border-left:4px solid #ffc107; border-radius:4px;">
                    <h4 style="margin:0; color:#856404;">📄 No Results Found</h4>
                    <p style="margin:5px 0; color:#856404;">
                        No research papers found for: "<strong>{q}</strong>"
                    </p>
                </div>
                """))
                status.value = "<div class='se-status' style='color:#d93025'>❌ No papers found</div>"
                return

            # Render: results header showing the query and count
            display(HTML(f"""
            <div style="margin-top:6px; margin-bottom:15px; color:#5f6368; font-size:13px">
              <b style="color:#202124">📚 Research Papers for:</b> "{q}"
              <span style="color:#5f6368">({len(results)} papers found)</span>
            </div>
            """))

            # Render: one card per paper (ranked order from search_documents)
            for i, result in enumerate(results, start=1):
                display(HTML(f"""
                <div class="se-results-card">
                    <div style="margin-bottom:8px;">
                        <span style="color:#5f6368; font-size:12px; font-weight:600;">#{i}</span>
                    </div>
                    <h4 style="margin:0 0 8px 0; color:#1a73e8;">
                        <a href="{result.get('url','')}" target="_blank" style="text-decoration:none; color:#1a73e8;">
                            {result.get('title','(no title)')}
                        </a>
                    </h4>
                    <div style="margin-bottom:8px;">
                        <span style="color:#5f6368; font-size:13px; font-weight:500;">
                            {result.get('domain','')}
                        </span>
                    </div>
                    <p style="margin:0; color:#3c4043; line-height:1.5; font-size:14px;">
                        {result.get('summary','')}
                    </p>
                </div>
                """))

        # UI state: update status after rendering
        status.value = f"<div class='se-status' style='color:#0a7'>✅ Found {len(results)} papers</div>"

    def on_clear(_):
        """Handle Clear click: reset query box, reset status line, and clear rendered results."""
        query.value = ""
        status.value = "<div class='se-status'>🔍 Ready to search.</div>"
        with out_results:
            clear_output(wait=True)

    # Event wiring: connect button clicks to handlers
    btn_search.on_click(on_search)
    btn_clear.on_click(on_clear)

    # Widget: action buttons row (HBox)
    actions = widgets.HBox([btn_search, btn_clear], layout=widgets.Layout(width="100%", gap="12px"))
    actions.add_class("se-actions")

    # Widget: example queries helper box (HTML)
    examples_html = widgets.HTML("""
    <div style="margin:10px 0; padding:10px; background:#f8f9fa; border-radius:4px;">
        <strong>💡 Example Queries:</strong><br/>
        <span style="font-size:12px; color:#5f6368;">
            Try: "machine learning detection", "deep research", "precision agriculture"
        </span>
    </div>
    """)

    # Widget: query container (VBox) so CSS targets textarea reliably
    query_box = widgets.VBox([query], layout=widgets.Layout(width="100%"))
    query_box.add_class("se-query")

    # Widget: main panel container (VBox)
    panel = widgets.VBox(
        [
            title,
            widgets.HTML("<b>Query</b>"),
            query_box,
            actions,
            examples_html,
            status,
            out_results
        ],
        layout=widgets.Layout(width="100%", max_width="980px", padding="20px")
    )
    panel.add_class("se-panel")

    return panel


#Display PurrFarm dashborad



In [ ]:
# Widget: app title container (HTML)
display(HTML("""
    <div class="cat-container">
      <div class="cat-header">
        <div class="cat-header-title">PurrFarm</div>
      </div>
    </div>
  """))

# Widget: Tab 1 output (Upload Image)
tab1_upload = widgets.Output()
with tab1_upload:
    # Widget: upload form UI (VBox)
    upload_form = create_plant_upload_form()
    display(upload_form)

# Widget: Tab 2 output (IoT Sensors)
tab2_iot = widgets.Output()
with tab2_iot:
    # Widget: IoT sensors UI (VBox)
    iot_interface = create_iot_sensors_tab()
    display(iot_interface)

# Widget: Tab 3 output (Analysis history)
tab3_ai = widgets.Output()
with tab3_ai:
    # Widget: history grid HTML (HTML)
    display(render_ai_analysis_tab())

# Widget: Tab 4 output (Dashboard)
tab4_dashboard = widgets.Output()
with tab4_dashboard:
    # Widget: dashboard UI (VBox)
    display(create_dashboard_tab())

# Widget: Tab 5 output (Search Engine)
tab5_gamification = widgets.Output()
with tab5_gamification:
    # Widget: search engine panel (VBox)
    display(build_plant_disease_search_widget())

# Widget: tabs container (Tab)
tabs = widgets.Tab(children=[
    tab1_upload, tab2_iot, tab3_ai,
    tab4_dashboard, tab5_gamification
])

# Tab labels (titles)
tabs.set_title(0, "📷 Upload Image")
tabs.set_title(1, "🌡️ IoT Sensors")
tabs.set_title(2, "📜 Analysis history")
tabs.set_title(3, "📊 Dashboard")
tabs.set_title(4, "🔎 Search Engine")

# Styling: attach the main tabs class and load CSS theme
tabs.add_class("custom-tabs")
load_cat_css()

# show the tabs in the notebook
display(tabs)
